<a href="https://colab.research.google.com/github/Bupashen/pdc/blob/main/LAB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu

#include <assert.h>
#include <stdio.h>
#include <chrono>
#include <math.h>
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/reduce.h>

//CUDA runtime
#include <cuda_runtime.h>

// Helper functions and utilities to work with CUDA




#ifndef __CUDACC__ 
#define __CUDACC__
#endif

#include <device_functions.h>
//Кол-во потоков
#define N 512
//Кол-во блоков
#define BL 32
//Не корень
#define NOTROOT -100
//Максимальное число итераций на интервале
#define N_MAX 51

//Интервал в границах поиска корней
#define A -100
#define B 100

//Точность
#define EPS 0.000001f

//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU//GPU

__device__ __host__ float f(float x)  //функция
{
	return   2.33 * sin(2.86 * x) - 2 * x;
}

__device__ __host__ float f1(float x)    // первая производная функции, f'
{
	return  2.33 * 2.86 * cos(2.86 * x) - 2;
}

__device__ __host__ float f2(float x)    //вторая производная функции, f''
{
	return -2.33 * 2.86 * 2.86 * sin(2.86 * x);
}

__global__ void NewtonsMethod(float* result_d, float step)
{

	//Считаем границы для gpu
  int idx = threadIdx.x + blockIdx.x * blockDim.x;
	float a = A + idx * step;
	float b = A + (idx + 1) * step;

	if (f(a) * f(b) * 1.0 > 0)
		return;  

  float c_prev = 0;
  float c_cur = 0;
  float c_next = 0;

	if (f(a) * f2(a) > 0)
		c_cur = a;
	else
		c_cur = b;  

	for(int j = 0; j < N_MAX - 1; j++)
  {
    c_next = c_cur - f(c_cur) / f1(c_cur);
    if (j >= 1)
    {
      if (fabs((c_next - c_cur) / (1 - (c_next - c_cur) / (c_cur - c_prev))) < EPS)
      {
        break;
      }
    }
    c_prev = c_cur;
    c_cur = c_next;
  }
 
  if (c_next <= b && c_next >= a)
  {
    result_d[idx] = c_next;
  }
}

int main(int argc, char* argv)
{
	size_t free, total;
	printf("\n");
	cudaMemGetInfo(&free, &total);
	printf("%d KB free of total %d KB\n", free / 1024, total / 1024);

	//Создаем массив для вывода корней(результата)
	const unsigned int n = N * BL;


	//thrust вектор для вывода корней
	thrust::host_vector<float> h(n, NOTROOT);
	thrust::device_vector<float> d(n, NOTROOT);


	float* result = thrust::raw_pointer_cast(d.data());

	//Вычисляем шаг для границ поиска корней
	float step = fabs(A - B) * 1.0 / n;

	////---ВЫЧИСЛЕНИЕ НА ВИДЕОКАРТЕ---
	printf("[Reverse computing Using CUDA] - Starting...\n");

	cudaStream_t stream;

	cudaDeviceSynchronize();

	//// Создание событий и стрима для таймера
	cudaEvent_t start, stop;
	cudaEventCreate(&start);
	cudaEventCreate(&stop);

	cudaStreamCreateWithFlags(&stream, cudaStreamNonBlocking);


	printf("Computing result using CUDA Kernel...\n");

	//// Запись начала события
	cudaEventRecord(start, stream);

	//// Выполнение кода на видеокарте и ожидание завершения всех потоков
	NewtonsMethod << <BL, N >> > (result, step);

	cudaStreamSynchronize(stream);

	//// Запись окончания события
	cudaEventRecord(stop, stream);

	//// Синхронизация и ожидание завершающего события
	cudaEventSynchronize(stop);

	//// Расчет и вывод производительности

	float m_sec_total = 0.0f;
	cudaEventElapsedTime(&m_sec_total, start, stop);

	float mc_sec_total = m_sec_total * 1000;
	printf(
		"Time GPU = %.10f microsec\n",
		mc_sec_total);

	cudaStreamSynchronize(stream);

	//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU//CPU

	// Начинаем замер времени
	auto begin = std::chrono::high_resolution_clock::now();

	for (unsigned int i = 0; i < n; i++)
	{
		//Вычисляем границы для CPU
		float a = A + i * step;
		float b = A + (i + 1) * step;

		if (f(a) * f(b) * 1.0 > 0)
			continue;

    float c[N_MAX];
    int k = N_MAX - 2;

	  if (f(a) * f2(a) > 0)
	  	c[0] = a;
	  else
	  	c[0] = b;

    for(int j = 0; j < N_MAX - 1; j++)
    {
      c[j + 1] = c[j] - f(c[j]) / f1(c[j]);
      if (j >= 1)
      {
        if (fabs((c[j + 1] - c[j]) / (1 - (c[j + 1] - c[j]) / (c[j] - c[j - 1]))) < EPS)
        {
          k = j + 1;
          break;
        }
      }
    }

    if (c[k] <= b && c[k] >= a)
    {
      h[i] = c[k];
    }	
	}

	////// Останавливаем таймер и считаем время выполнения
	auto end = std::chrono::high_resolution_clock::now();
	auto elapsed = std::chrono::duration_cast<std::chrono::nanoseconds>(end - begin);

	printf(
		"Time CPU = %.10f microsec\n",
		elapsed.count() * 1e-3);

	//Вывод значений

  for (int i = 0; i < d.size(); i++)
  {
    if (d[i] != NOTROOT)
    {
      printf("GPU root (%d) == %f \n", i, (float)d[i]);
    }
  }

  for (int i = 0; i < h.size(); i++)
  {
    if (h[i] != NOTROOT)
    {
      printf("CPU root (%d) == %f \n", i, (float)h[i]);
    }
  }
	//// Освобождение памяти
	cudaEventDestroy(start);
	cudaEventDestroy(stop);

	

	return 0;
}


15365888 KB free of total 15472384 KB
[Reverse computing Using CUDA] - Starting...
Computing result using CUDA Kernel...
Time GPU = 8.2880001068 microsec
Time CPU = 980.5470000000 microsec
GPU root (8124) == -0.823825 
GPU root (8191) == 0.000000 
GPU root (8192) == 0.000000 
GPU root (8259) == 0.823825 
CPU root (8124) == -0.823825 
CPU root (8191) == 0.000000 
CPU root (8192) == 0.000000 
CPU root (8259) == 0.823825 

